In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os


## Data parser

Find and download all microrheology data and format dictionary for stan

In [4]:
# Download data and make a mega .csv fiel

paths=[]
paths = glob.glob(os.path.join('C:/Users/srboval1/IPN15/collagen/*','**','summary_ref_level.csv'))
print(paths)
concentration = '0/2'
type ='collagen'

a = []
for i in paths:
    path1 = os.path.split(os.path.split(i)[0])[0]
    path1.replace("/","\\")
    splitted= path1.split('\\')[-1]
    tmp = pd.read_csv(i)
    tmp['coating_type'] = splitted.split('_')[-1]
    tmp['size'] = splitted.split('_')[-2]
    tmp['day'] = splitted.split('_')[-4]
    a.append(tmp)

a_concantenated=[]
a_concantenated = pd.concat(a)
a_concantenated['radius_(m)'] *= 1e6
a_concantenated = a_concantenated.rename(columns={'radius_(m)':'radius_(um)'})
a_concantenated['concentration'] = concentration
a_concantenated['type'] = type
a_concantenated['frequency'] = 0.05
a_concantenated = a_concantenated.reindex(columns=['day','frequency','concentration','type','sample','holder','location','repeat','track_id','reference_id','distance(um)','Cov_Sum','a_(um)','phi_(rad)','c','d','G_abs','radius_(m)','r2','rmse','inv.rmse','shift_(s)','a_error','phi_error','c_error','d_error','x','y','phi_(deg)','tan_phi'])
print(a_concantenated)
#%%
if os.path.exists('C:/Users/srboval1/IPN15/IPN15.csv'):
    a_new = pd.DataFrame(a_concantenated)
    a_new.to_csv('C:/Users/srboval1/IPN15/IPN15.csv', mode='a', index=False, header=False)
else:
    a_concantenated.to_csv("C:/Users/srboval1/IPN15/IPN15.csv", index=False)
    

['C:/Users/srboval1/IPN15/collagen\\211029_0_10_plain\\results\\summary_ref_level.csv']
       day  frequency concentration      type  sample  holder  location  \
0   211029       0.05           0/2  collagen      10      10        10   
1   211029       0.05           0/2  collagen      10      10        10   
2   211029       0.05           0/2  collagen      10      10        10   
3   211029       0.05           0/2  collagen      10      10        10   
4   211029       0.05           0/2  collagen      10      10        10   
5   211029       0.05           0/2  collagen      10      10        10   
6   211029       0.05           0/2  collagen      10      10        10   
7   211029       0.05           0/2  collagen      10      10        10   
8   211029       0.05           0/2  collagen      10      10        10   
9   211029       0.05           0/2  collagen      10      10        10   
10  211029       0.05           0/2  collagen      10      10        10   
11  211029  

In [10]:
data_all = pd.read_csv('C:/Users/srboval1/IPN15/IPN15.csv')

#calculating G std for each material typeand then choosing the highest one of all
G_std = np.max(data_all[['type','day','holder','location','track_id',"G_abs", "phi_(deg)"]].groupby(["type"]).std()["G_abs"].values)
print(G_std)

#calculating G std for each material type and then choosing the highest one of all
phi_std = np.max(data_all[['type','day','holder','location','track_id',"G_abs", "phi_(deg)"]].groupby(["type"]).std()["phi_(deg)"].values)
print(phi_std)


full_path = os.path.split(os.getcwd())[0]

print(["G_abs", "phi_(rad)"], "\n",data_all["type"].unique())

3880.8730321837993
10.763319073259654
['G_abs', 'phi_(rad)'] 
 ['IPN soft' 'IPN stiff' 'collagen']


Make a dictionary

In [22]:
#%%
data_all = pd.read_csv('C:/Users/srboval1/IPN15/IPN15.csv')

#%%

#process mechanical properties separately, 
#for the final analysis it measures either G or phi
for measurement in ['G_abs','phi_(deg)']:
    samples = []

    #combine information of all materials
    #looping through each material one by one
    for material in data_all["type"].unique():
        data_type = material
        data = data_all[data_all['type'] == data_type]

        #initializing empty list that will contain corresponding numbers for each sample
        sample_num = []
        
        prev = None
        idx = 1
        #assuming one sample per day, adding sample ids to the list in sample_num
        #iterating over each element in the 'day' column of data and creating the sample_num list
        for i in data['day']:
            if prev is None:
                prev = i
            else: #changed from Ossi's code 'if' - to have distinct sample ids
                idx += 1
                prev = i
            sample_num.append(idx)      
        data['sample_num'] = sample_num
        subset = data [['sample_num','holder','location', 'track_id', measurement]]
        subset['sample_num']=subset['sample_num'].astype(int) #convertin sample numbers to integers
        
        sample_ids=[]
        location_ids=[]
        holder_ids=[]
        track_ids=[]

        n_loc = 0       #initial number of ids in locations
        n_hold = 0      #initial locations in holders
        n_samp = 0    #initial holders in samples

    #change into consecutive numbering
        for i in subset['sample_num'].unique():             
            sub_sample = subset[subset['sample_num']==i]        #new df containing only the rows (measurements) with that unique sample_num i
            sample_ids.extend([int(i)]*sub_sample['holder'].unique().shape[0])
            for jj,j in enumerate(sub_sample['holder'].unique()):
                sub = sub_sample[sub_sample['holder']==j]
                holder_ids.extend([int(jj+1)+n_samp]*sub['location'].unique().shape[0]) #list that will eventually be listing all holder ids for each location - add identity number of holder times the number of locations (unique holder numbers)
                for kk,k in enumerate(sub['location'].unique()):
                    sub2 = sub_sample[sub_sample['location']==j]
                    location_ids.extend([int(kk+1)+n_hold]*sub['track_id'].unique().shape[0])
                    for zz, z in enumerate(sub2['track_id'].unique()):
                        sub3 = sub2[sub2['track_id']==z]
                        track_ids.extend([int(zz+1)+n_loc]*sub3.shape[0]) #shape[0] size of the first dimension = row
                    n_loc += int(sub2['track_id'].unique().shape[0])    #number of tracks ids that have been looped through already; updated with each location
                n_hold += int(sub['location'].unique().shape[0])    #number of locations that have been looped through; used to add up in the next holder
            n_samp += int(sub_sample['holder'].unique().shape[0])   #number of holders that have been looped through; used to add up in the next sample

    # normalize mean to zero and with common std 
    y_raw = subset[measurement].values  #retrieves values of the column labelled measurement from df subset and converts into array by .value
    if measurement == 'G_abs':          #compares measurement to a fixed string 'G_abs' variable
        y = y_raw/G_std                 #each value scaled by std
    else:
        y = y_raw/phi_std

    #defining dictionary; curly brackets and specified key-value pairs separated by : each key corresponds to a specific attribute }
    sample = {    'N':subset.shape[0],
                    'N_samples':  int(np.max(sample_ids)),
                    'N_holders':  int(np.max(holder_ids)),
                    'N_locations':int(np.max(location_ids)),
                    'N_ids':      int(np.max(track_ids)),
                    'sample_ids': sample_ids,
                    'holder_ids': holder_ids,
                    'location_ids':location_ids,
                    'track_ids':  track_ids,
                    'train_ids':  (np.arange(len(track_ids))+1).tolist(),
                    'N_train':    subset.shape[0],
                    'y':          y.tolist()}
    
    #appending a shallow copy 'samples' of sample - doesn't create copies of the objects it references to
    samples.append(sample.copy())

# combine different materials into a single dict/json for stan
combined = samples[0]           #combined is assigned the value of the first element (samples[0]) from the list samples
combined['material_ids'] = np.ones(combined['N_samples'],dtype=int).tolist() #creates a list of ones with a length determined by the value of 'N_samples'
s_max = combined['N_samples']
h_max = combined['N_holders']
l_max = combined['N_locations']
t_max = combined['N_ids']
m_max = 1

for s in samples[1:]:       #code iterates over all elements in the samples list except the first one
    #print(s['N'])
    combined['N'] += s['N'] #accesses the value associated with the key 'N' in the dictionary combined and incrementing it by the value associated with the key 'N' in each dictionary s in the samples list
    combined['N_samples'] += s['N_samples']
    combined['N_holders'] += s['N_holders']
    combined['N_locations'] += s['N_locations']
    combined['N_ids'] += s['N_ids']
    combined['sample_ids'].extend((np.array(s['sample_ids'])+s_max).tolist())
    combined['holder_ids'].extend((np.array(s['holder_ids'])+h_max).tolist())
    combined['location_ids'].extend((np.array(s['location_ids'])+l_max).tolist())
    combined['track_ids'].extend((np.array(s['track_ids'])+t_max).tolist())
    combined['y'].extend(s['y'])
    combined['material_ids'].extend((np.ones(s['N_samples'],dtype=int)+m_max).tolist())

    s_max += s['N_samples']
    h_max += s['N_holders']
    l_max += s['N_locations']
    t_max += s['N_ids']
    m_max += 1

combined['train_ids'] = ((np.arange(combined['N'])+1).astype(int)).tolist()
combined['N_train'] = len(combined['train_ids'])
combined['sample_ids'] = combined['sample_ids']
combined['holder_ids'] = combined['holder_ids']
combined['location_ids'] = combined['location_ids']
combined['track_ids'] = combined['track_ids']
combined['material_ids'] = np.array(combined['material_ids']).astype(int).tolist()
combined['material_ids'] = np.array(combined['material_ids']).astype(int).tolist()
combined['N_materials'] = len(samples)

import json

#crating path to save the dictionary in json format
file_path = "C:/Users/srboval1/IPN15/IPN15_dict.json"

# Writing the dictionary to a JSON file
with open(file_path, "w") as json_file:
    json.dump(combined, json_file)



C:\Users\srboval1\AppData\Local\Temp\ipykernel_17808\1473737976.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sample_num'] = sample_num
C:\Users\srboval1\AppData\Local\Temp\ipykernel_17808\1473737976.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['sample_num']=subset['sample_num'].astype(int) #convertin sample numbers to integers
C:\Users\srboval1\AppData\Local\Temp\ipykernel_17808\1473737976.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da